In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

import datetime as dt
import plotly.graph_objects as go

from tqdm import tqdm

In [2]:
data = pd.read_parquet("stock_data.parquet")
stock_names = data['comnam'].unique()

# repeat = True
# while repeat:
#     count = 0
#     dfs = []
#     sizes = []
#     relevant_stocks = np.random.choice(stock_names,size=10, replace=False)
#     for stock in relevant_stocks:
#         subset = data[data['comnam'] == stock]
#         df = subset[['prc']]
#         df.columns = [stock]
#         df.index = pd.to_datetime(subset['date'])
#         dfs.append(df)
#         sizes.append(df.shape[0])
#     dfs = [x for _,x in reversed(sorted(zip(sizes,dfs), key = lambda x:x[0]))]
#     final = dfs[0]
#     for i in range(1, 10):
#         final = final.join(dfs[i])
#     final.index.name = 'date'
#     final = final.groupby(by='date').mean()
#     final = final.interpolate()
#     period = np.random.randint(30, final.shape[0])
#     final = final.iloc[period-30:period]
#     if final.iloc[0].isna().any() or final.iloc[29].isna().any():
#         repeat = True
#     else:
#         repeat = False

In [3]:
data

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,bid,ask,shrout,cfacpr,cfacshr,openprc,numtrd,retx,ticker,comnam
33229,68389X10,10104.0,8045.0,10536.0,1.0,7372.0,2020-01-02,53.230000,53.959999,53.950001,...,53.950001,53.959999,3207649.0,1.0,1.0,53.270000,NaN,0.018309,ORCL,ORACLE SYSTEMS CORP
33230,68389X10,10104.0,8045.0,10536.0,1.0,7372.0,2020-01-03,52.950001,54.049999,53.759998,...,53.770000,53.779999,3207649.0,1.0,1.0,52.990002,NaN,-0.003522,ORCL,ORACLE SYSTEMS CORP
33231,68389X10,10104.0,8045.0,10536.0,1.0,7372.0,2020-01-06,53.349998,54.200001,54.040001,...,54.040001,54.049999,3207649.0,1.0,1.0,53.360001,NaN,0.005208,ORCL,ORACLE SYSTEMS CORP
33232,68389X10,10104.0,8045.0,10536.0,1.0,7372.0,2020-01-07,53.610001,54.330002,54.160000,...,54.160000,54.169998,3207649.0,1.0,1.0,53.889999,NaN,0.002221,ORCL,ORACLE SYSTEMS CORP
33233,68389X10,10104.0,8045.0,10536.0,1.0,7372.0,2020-01-08,53.700001,54.599998,54.130001,...,54.119999,54.130001,3207649.0,1.0,1.0,53.939999,NaN,-0.000554,ORCL,ORACLE SYSTEMS CORP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340541,88160R10,93436.0,53453.0,66252.0,3.0,9999.0,2022-03-25,997.320129,1021.799927,1010.640015,...,1010.210022,1010.890015,1033534.0,1.0,1.0,1008.000000,593068.0,-0.003235,TSLA,TESLA INC
340542,88160R10,93436.0,53453.0,66252.0,3.0,9999.0,2022-03-28,1053.599976,1097.879883,1091.839966,...,1091.839966,1092.199951,1033534.0,1.0,1.0,1065.099976,1113132.0,0.080345,TSLA,TESLA INC
340543,88160R10,93436.0,53453.0,66252.0,3.0,9999.0,2022-03-29,1073.109985,1114.770020,1099.569946,...,1099.819946,1099.959961,1033534.0,1.0,1.0,1107.989990,835619.0,0.007080,TSLA,TESLA INC
340544,88160R10,93436.0,53453.0,66252.0,3.0,9999.0,2022-03-30,1084.000000,1113.949951,1093.989990,...,1093.689941,1093.989990,1033534.0,1.0,1.0,1091.170044,598381.0,-0.005075,TSLA,TESLA INC


In [4]:
# Pivot data (columns: stock name, index: date)
data_pivot = data.pivot_table(index="date", values="prc", columns="comnam")

# Remove stocks with na in price column random stocks
# TODO: Remove the complete stock if any na
data_clean = data_pivot.dropna(axis=1, how='any')
data_clean
# # Pick 10 stocks at random
# relevant_stocks = np.random.choice(data_clean.columns,size=10, replace=False)
# data_filt = data_clean[relevant_stocks]

# # Focus on window of 30 trading days with random start dt
# start_dt = np.random.choice(data_filt.index[:-30],size=1, replace=False)[0]
# final = data_pivot.query("date >= @start_dt")
# final = final.iloc[:30]

# final
# data_filt

comnam,10X GENOMICS INC,3M CO,A C C O BRANDS CORP,A D T INC,A E S CORP,A H BELO CORP,A M B PROPERTY CORP,A M N HEALTHCARE SERVICES INC,A M R CORP DEL,A P A CORP,...,ZAYRE CORP,ZEBRA TECHNOLOGIES CORP,ZILLOW GROUP INC,ZILLOW INC,ZIMMER BIOMET HOLDINGS INC,ZIMMER HOLDINGS INC,ZIONS BANCORPORATION,ZIONS BANCORPORATION N A,ZIONS UTAH BANCORPORATION,ZOETIS INC
date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,72.830002,180.000000,9.11,7.57,20.030001,2.830,88.400002,62.110001,29.090000,25.360001,...,61.700001,259.140015,45.157499,45.000000,149.199997,149.199997,52.439999,52.439999,52.439999,134.139999
2020-01-03,75.559998,178.449997,8.99,7.45,19.799999,2.820,89.389999,62.419998,27.650000,25.690001,...,61.240002,256.049988,44.680001,44.560001,148.809998,148.809998,51.700001,51.700001,51.700001,134.160004
2020-01-06,75.550003,178.619995,9.08,7.56,20.030001,2.920,89.040001,62.110001,27.320000,25.639999,...,61.630001,258.010010,44.587500,44.430000,147.949997,147.949997,51.070000,51.070000,51.070000,133.130005
2020-01-07,75.980003,177.899994,8.87,7.44,20.100000,2.910,88.470001,62.049999,27.219999,32.509998,...,61.590000,256.470001,45.697500,45.480000,147.820007,147.820007,50.759998,50.759998,50.759998,133.580002
2020-01-08,74.839996,180.630005,8.84,6.81,20.059999,2.891,88.949997,63.490002,27.840000,32.730000,...,61.919998,247.639999,45.672501,45.380001,149.589996,149.589996,51.259998,51.259998,51.259998,133.289993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-25,67.790001,150.460007,8.19,7.36,24.370001,6.940,157.330002,102.250000,17.010000,41.529999,...,61.290001,425.660004,51.742500,50.849998,125.279999,125.279999,71.080002,71.080002,71.080002,189.429993
2022-03-28,70.410004,149.470001,8.19,7.45,24.650000,7.030,160.350006,105.510002,17.299999,39.849998,...,62.000000,422.279999,52.474999,51.500000,125.680000,125.680000,69.730003,69.730003,69.730003,189.369995
2022-03-29,77.250000,152.080002,8.34,7.85,25.450001,6.970,164.699997,106.440002,18.160000,40.959999,...,64.309998,438.100006,55.107499,54.110001,129.580002,129.580002,69.739998,69.739998,69.739998,192.279999


Create an initial portfolio

In [5]:
def get_prices_window(prices, start_dt):
    # Pick 10 stocks at random
    relevant_stocks = np.random.choice(prices.columns,size=10, replace=False)
    prices_filt = prices[relevant_stocks]

    final = prices_filt.query("date >= @start_dt")
    final = final.iloc[:30]
    return final

def generate_portfolios(stock_prices, init_budget=100000):
    initial_budget = init_budget
    initial_prices = stock_prices.iloc[0].to_numpy()
    # available_money = initial_budget * .95
    available_money = initial_budget
    P_0 = np.zeros(10) #initial portfolio
    break_bool = True
    ignore =  np.random.randint(0,10)
    while break_bool:
        stock_buy = np.random.randint(0,10)
        price = initial_prices[stock_buy]
        if stock_buy != ignore:
            if (available_money * .75) - price >= 0:
                available_money = available_money - price
                P_0[stock_buy] = P_0[stock_buy] + 1
            else:
                break_bool = False

    # w_0 = initial_budget * .05 + available_money
    w_0 = available_money
    # print(w_0)
    # print(P_0)
    final_prices = stock_prices.iloc[29].to_numpy()
    available_money = initial_budget * .9
    break_bool = True
    P_t = np.zeros(10) #target portfolio

    while break_bool:
        stock_buy = np.random.randint(0,10)
        price = final_prices[stock_buy]
        if available_money - price >= 0:
            available_money = available_money - price
            P_t[stock_buy] = P_t[stock_buy] + 1
        else:
            break_bool = False
    # print(P_t)
    return P_0, P_t, w_0, initial_prices, final_prices

# Formulations

## With Perfect Information

In [6]:
def get_naive_opt(P_0, P_t, w_0, initial_budget, initial_prices, final_prices, print_bool=False):
    F = 5.95 * np.ones(10)# trading fees
    # P_0 = starting portfolio
    # P_t = target portfolio
    # w_0 = starting cash
    # w_f = final free cash
    # budget = 20000
    M = initial_budget

    # Silence Gurobi output
    env = gp.Env(empty=True)
    env.setParam("OutputFlag",0)
    env.start()

    m = gp.Model(env=env)
    # Define Variables
    P_buy = m.addMVar((10,), vtype=GRB.INTEGER)
    P_sell = m.addMVar((10,), vtype=GRB.INTEGER)
    z_sell = m.addMVar((10,), vtype=GRB.BINARY)
    z_buy = m.addMVar((10,), vtype=GRB.BINARY)
    # Constraints

    ## Meet Target
    m.addConstr(
        P_0 + P_buy - P_sell >= P_t
    )

    ## Have Money
    m.addConstr(
        w_0 + initial_prices @ P_sell - initial_prices @ P_buy - F @ z_sell - F @ z_buy >=0
    )

    ## if we sell a stock, we pay a trading price
    m.addConstr(
        M*(z_sell) >= P_sell
    )

    ## If we buy a stock, we pay a trading fee

    m.addConstr(
        M*(z_buy) >= P_buy
    )

    cash = w_0 + initial_prices @ P_sell - initial_prices @ P_buy
    # Set objective: Maximize end portfolio value
    m.ModelSense = GRB.MAXIMIZE
    # try:
    m.setObjective(
        # cash - F @ z_sell - F @ z_buy + final_prices @ (P_0 + P_buy - P_sell)
        cash + final_prices @ (P_0 + P_buy - P_sell)
    )
    # except Exception as e:
        # print(e)
        # print((cash - F @ z_sell - F @ z_buy + final_prices @ (P_0 + P_buy - P_sell)).getValue())
        # print(cash.x)
        # print(z_sell.x)
        # print(z_buy.x)
        # print(final_prices)
        # print(P_0.x)
        # print(P_buy.x)
        # print(P_sell.x)

    m.optimize()
    final_portfolio = P_0 + P_buy.x - P_sell.x
    # free_cash = w_0 + initial_prices @ P_sell.x - initial_prices @ P_buy.x
    cash = cash.getValue()
    final_portfolio_value = cash + final_prices @ final_portfolio
    
    if print_bool:
        print(f"Init Portf Value: {initial_prices @ P_0}")
        print(f"Final Portfolio Value: {final_portfolio_value}")
        print(f"Cash Init: {w_0}")
        print(f"Cash Final: {cash}")
        print(f"Init Portfolio: {P_0}")
        print(f"Final Portfolio: {final_portfolio}")
        print(f"Target Portfolio: {P_t}")
        print(f"TX Vec Buy: {z_buy.x}")
        print(f"TX Vec Sell: {z_sell.x}")
        print(f"Total Tx Costs: {F @ z_sell.x + F @ z_buy.x}")
    return final_portfolio_value, final_portfolio, cash

In [7]:
# test
rng = np.random.default_rng()
budget = rng.integers(low=1000, high=10000, size=1)[0]
start_dt = np.random.choice(data_pivot.index[:-30], size=1, replace=True)[0]
stock_price_window = get_prices_window(data_clean, start_dt)
P_0, P_t, w_0, initial_prices, final_prices = generate_portfolios(stock_prices = stock_price_window, init_budget=budget)
final_portfolio_value, final_portfolio, free_cash = get_naive_opt(P_0, P_t, w_0, budget, initial_prices, final_prices, print_bool=True)

Init Portf Value: 3352.129997253418
Final Portfolio Value: 3746.1300048828125
Cash Init: 86.87000274658203
Cash Final: 61.799983978271484
Init Portfolio: [5. 3. 4. 3. 0. 4. 3. 3. 1. 0.]
Final Portfolio: [ 1.  1.  2.  3. 10.  2.  5.  4.  0.  2.]
Target Portfolio: [1. 1. 2. 3. 5. 0. 5. 4. 0. 2.]
TX Vec Buy: [0. 1. 0. 0. 1. 0. 1. 1. 0. 1.]
TX Vec Sell: [1. 1. 1. 0. 0. 1. 0. 0. 1. 0.]
Total Tx Costs: 59.5


## Multi-stage

##

In [8]:
def get_multi_stage_opt(P_0, P_t, w_0, initial_budget, stock_price_window, print_bool=False):   
    # Silence Gurobi output
    env = gp.Env(empty=True)
    env.setParam("OutputFlag",0)
    env.start()

    F = 5.95  * np.ones(10)# trading fees
    # prices = stock_price_window.interpolate().to_numpy()
    prices = stock_price_window.to_numpy()
    M = initial_budget
    # P_0 = starting portfolio
    # P_t = target portfolio
    # w_0 = starting cash

    m = gp.Model(env=env)

    # Define Variables
    P_buy = m.addMVar((30,10), vtype=GRB.INTEGER)
    P_sell = m.addMVar((30,10), vtype=GRB.INTEGER)
    z_sell = m.addMVar((30,10), vtype=GRB.BINARY)
    z_buy = m.addMVar((30,10), vtype=GRB.BINARY)
    P_time = m.addMVar((30,10), vtype=GRB.INTEGER)
    # P_time = np.ndarray((30,10))
    w = m.addMVar((30,), vtype=GRB.CONTINUOUS, lb = 0)
    # w = np.ndarray((30,))
    # Constraints

    ## Represent Portfolios across time

    ### Time step 0
    m.addConstr(
        P_time[0,:] == P_0 + P_buy[0,:] - P_sell[0,:]
    )
    ### Rest of the time
    m.addConstrs((
        P_time[i,:] == P_time[i-1,:] - P_buy[i,:] - P_sell[i,:] for i in range(1,30)
    ))

    ### Final_target
    m.addConstr(
        P_time[29,:] >= P_t
    )

    ## Represent money
    ### First time step
    m.addConstr(
        w[0] == w_0 - prices[0,:] @ P_buy[0,:] + prices[0,:] @ P_sell[0,:] - F @ z_buy[0,:] - F @ z_sell[0,:]
    )
    ## Rest of the time
    m.addConstrs((
        w[i] == w[i-1] - prices[i,:] @ P_buy[i,:] + prices[i,:] @ P_sell[i,:] - F @ z_buy[i,:] - F @ z_sell[i,:] for i in range(1,30)
    ))

    ## Trading fees
    ### if we sell a stock, we pay a trading price
    m.addConstr(
        M*(z_sell) >= P_sell
    )

    ### If we buy a stock, we pay a trading fee
    m.addConstr(
        M*(z_buy) >= P_buy
    )

    # Maximize end Portfolio value
    m.ModelSense = GRB.MAXIMIZE
    m.setObjective(
        # w[29] + prices[29,:] @ P_time[29,:]
        prices[29,:] @ P_time[29,:]
    )
    m.optimize()
    final_portfolio = P_time[29,:].x
    cash = w[29].x 
    final_portfolio_value = cash + prices[29,:] @ final_portfolio
    # print(f"Final Portfolio Value: {final_portfolio_value}")
    # print(f"Free Cash: {free_cash}")
    # print(f"Final Portfolio: {final_portfolio}")
    if print_bool:
        print(f"Init Portf Value: {initial_prices @ P_0}")
        print(f"Final Portfolio Value: {final_portfolio_value}")
        print(f"Cash Init: {w_0[0]}")
        print(f"Cash Final: {cash}")
        print(f"Init Portfolio: {P_0}")
        print(f"Final Portfolio: {final_portfolio}")
        print(f"Target Portfolio: {P_t}")
        print(f"TX Vec Buy: {z_buy.x}")
        print(f"TX Vec Sell: {z_sell.x}")
        print(f"Total Tx Costs: {sum([F @ z_sell.x[i] + F @ z_buy.x[i] for i in range(30)])}")
    return final_portfolio_value, final_portfolio, free_cash

In [9]:
def get_multi_stage_opt_2(P_0, P_t, w_0, initial_budget, stock_price_window, print_bool=False, time_limit=None):   
    # Silence Gurobi output
    env = gp.Env(empty=True)
    env.setParam("OutputFlag",0)
    env.setParam("TimeLimit",time_limit)
    env.start()

    F = 5.95  * np.ones(len(P_0))# trading fees
    # F = 5.95
    # prices = stock_price_window.interpolate().to_numpy()
    prices = stock_price_window.to_numpy()
    M = initial_budget

    buy_list = []
    sell_list = []
    obj_arr = []
    z_b_list = []
    z_s_list = []
    p_val_list = []
    cash_list = []
    p_list = []

    # Define init vars
    P_curr = P_0
    cash_curr = w_0
    # P_time = m.addMVar((30,10), vtype=GRB.INTEGER)

    m = gp.Model(env=env)

    for i, prices in stock_price_window.iterrows():
        prices = prices.to_numpy()
        # m = gp.Model(env=env)

        # Define in loop vars
        # Define Variables
        P_buy = m.addMVar((len(P_0),), vtype=GRB.INTEGER)
        P_sell = m.addMVar((len(P_0),), vtype=GRB.INTEGER)
        z_sell = m.addMVar((len(P_0),), vtype=GRB.BINARY)
        z_buy = m.addMVar((len(P_0),), vtype=GRB.BINARY)

        # print(prices @ P_sell)

        # This is the next iteration of portfolio
        P_next = P_curr + P_buy - P_sell
        cash_next = prices @ P_sell - prices @ P_buy - F @ z_sell - F @ z_buy + cash_curr

        ## Trading fees
        ### if we sell a stock, we pay a trading price
        m.addConstr(
            M*(z_sell) >= P_sell
        )

        ### If we buy a stock, we pay a trading fee
        m.addConstr(
            M*(z_buy) >= P_buy
        )

        # No borrowing
        m.addConstr(cash_next >= 0)

        # No shorting
        m.addConstr(P_next >= 0)

        obj_arr.append(prices @ P_next - F @ z_sell - F @ z_buy)
        cash_curr = cash_next
        P_curr = P_next
        # prob_arr.append(m)

        p_val_list.append(prices @ P_next)
        buy_list.append(P_buy)
        sell_list.append(P_sell)
        z_b_list.append(z_buy)
        z_s_list.append(z_sell)
        cash_list.append(cash_next)
        p_list.append(P_next)


    # Terminal weights constraints
    m.addConstr(P_next >= P_t)

    # Combine all time instances
    obj = sum(obj_arr)
    m.setObjective(obj, GRB.MAXIMIZE)
    m.optimize()
    
    buy_arr = np.array([el.x for el in buy_list])
    sell_arr = np.array([el.x for el in sell_list])
    z_b_arr = np.array([el.x for el in z_b_list])
    z_s_arr = np.array([el.x for el in z_s_list])
    p_val_arr = np.array([el.getValue() for el in p_val_list])
    cash_arr = np.array([el.getValue() for el in cash_list])
    p_arr = np.array([el.getValue() for el in p_list])

    if print_bool:
        print(f"Init Portf Value: {initial_prices @ P_0}")
        print(f"Final Portfolio Value: {prices @ P_next.getValue()}")
        print(f"Cash Init: {w_0}")
        print(f"Cash Final: {cash_next.getValue()}")
        print(f"Init Portfolio: {P_0}")
        print(f"Final Portfolio: {P_next.getValue()}")
        print(f"Target Portfolio: {P_t}")
        print(f"TX Vec Buy: {z_b_arr}")
        print(f"TX Vec Sell: {z_s_arr}")
        print(f"Total Tx Costs: {sum([F @ z_s_arr[i] + F @ z_b_arr[i] for i in range(30)])}")

    return buy_arr, sell_arr, z_s_arr, z_b_arr, p_val_arr, cash_arr, p_arr

In [10]:
# test
rng = np.random.default_rng()
budget = rng.integers(low=1000, high=10000, size=1)
start_dt = np.random.choice(data_pivot.index[:-30], size=1, replace=True)[0]
stock_price_window = get_prices_window(data_clean, start_dt)
P_0, P_t, w_0, initial_prices, final_prices = generate_portfolios(stock_prices = stock_price_window, init_budget=budget)
final_portfolio_value, final_portfolio, free_cash = get_multi_stage_opt(P_0, P_t, w_0, budget, stock_price_window, print_bool=True)
buy_list, sell_list, z_s_list, z_b_list, p_val_arr, cash_arr, p_arr = get_multi_stage_opt_2(P_0, P_t, w_0, budget, stock_price_window, print_bool=True, time_limit=10)

Init Portf Value: 1337.0900149345398
Final Portfolio Value: 1308.8582280397416
Cash Init: 10.909985065460205
Cash Final: 0.7200035572051888
Init Portfolio: [3. 3. 1. 2. 0. 1. 1. 3. 1. 0.]
Final Portfolio: [1. 4. 2. 5. 1. 3. 1. 6. 6. 1.]
Target Portfolio: [1. 4. 2. 4. 1. 3. 1. 2. 3. 1.]
TX Vec Buy: [[ 0.  1.  1.  1.  1.  1.  0.  1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0. -0. -0.  0.  0.]
 [ 0.  0. -0.  0.  0.  0. -0. -0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -0. -0.  0. -0.]
 [ 0.  0.  0.  0.  0.  0. -0. -0. -0.  0.]
 [ 0.  0.  0.  0.  0.  0. -0. -0. -0.  0.]
 [ 0.  0.  0.  0.  0.  0. -0. -0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -0. -0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -0. -0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -0. -0.  0.  0.]
 [ 0.  0. -0.  0.  0.  0. -0. -0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -0. -0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -0. -0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -0. -0. -0.  0.]
 [ 0.  0.  0.  0. -0.  0. -0. -0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -0. -0.  0.  0.]
 [-0.  0.  0.

In [11]:
cash_arr

array([[7.80002642e+00],
       [4.86004457e+00],
       [4.86004457e+00],
       [4.99720573e-02],
       [1.90045929e-01],
       [1.90045929e-01],
       [5.81992359e+00],
       [2.64354001e+03],
       [7.44002912e+00],
       [7.44002912e+00],
       [9.45994241e+00],
       [2.91006486e+00],
       [6.69014528e+00],
       [5.37301172e+01],
       [1.37301759e+01],
       [1.64301668e+01],
       [2.57132998e+03],
       [4.04898231e+01],
       [9.69698585e+01],
       [5.02984903e+00],
       [2.97828436e-02],
       [2.44430757e-01],
       [8.34458983e+00],
       [9.46462653e+00],
       [1.70461447e+00],
       [6.82429831e+00],
       [6.80427268e+00],
       [3.62443793e+00],
       [1.18243353e+01],
       [3.74315095e-01]])

In [12]:
pd.DataFrame(data=p_arr, columns = stock_price_window.columns, index=stock_price_window.index)

comnam,WASHINGTON POST CO,DURCO INTERNATIONAL INC,UNIPAC CORP,COMPUGEN LTD,PHILLIPS 66,M G M MIRAGE,AMERICAS GOLD AND SILVER CORP,SECOND SIGHT MEDICAL PRODUCTS IN,INSPERITY INC,AMERIPRISE FINANCIAL INC
date,,,,,,,,,,
2020-06-01,0.0,0.0,0.0,84.0,0.0,1.0,1.0,3.0,0.0,0.0
2020-06-02,0.0,0.0,0.0,0.0,0.0,78.0,1.0,3.0,0.0,0.0
2020-06-03,0.0,0.0,0.0,0.0,0.0,78.0,1.0,3.0,0.0,0.0
2020-06-04,0.0,0.0,0.0,0.0,0.0,0.0,1.0,417.0,22.0,0.0
2020-06-05,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1708.0,0.0,0.0
2020-06-08,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1708.0,0.0,0.0
2020-06-09,0.0,0.0,0.0,155.0,0.0,0.0,1.0,361.0,0.0,0.0
2020-06-10,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2020-06-11,0.0,0.0,0.0,205.0,0.0,0.0,1.0,0.0,0.0,0.0


In [13]:
stock_price_window

comnam,WASHINGTON POST CO,DURCO INTERNATIONAL INC,UNIPAC CORP,COMPUGEN LTD,PHILLIPS 66,M G M MIRAGE,AMERICAS GOLD AND SILVER CORP,SECOND SIGHT MEDICAL PRODUCTS IN,INSPERITY INC,AMERIPRISE FINANCIAL INC
date,,,,,,,,,,
2020-06-01,358.600006,27.040001,72.589996,15.320000,78.830002,17.799999,2.74,1.0100,53.369999,143.940002
2020-06-02,360.170013,27.160000,72.080002,16.790001,81.570000,18.200001,2.58,1.0000,55.000000,146.399994
2020-06-03,370.869995,29.090000,72.510002,17.120001,83.730003,20.260000,2.47,1.0200,57.570000,151.479996
2020-06-04,374.279999,29.610001,70.919998,14.720000,83.129997,21.719999,2.54,1.0200,57.220001,153.119995
2020-06-05,386.730011,31.590000,72.230003,13.610000,87.400002,21.719999,2.45,1.0600,62.750000,155.570007
2020-06-08,407.570007,33.630001,70.400002,13.580000,89.730003,23.760000,2.50,1.2600,62.450001,162.699997
2020-06-09,389.799988,31.840000,70.430000,13.270000,85.669998,22.059999,2.46,1.5400,60.599998,159.589996
2020-06-10,372.920013,30.129999,70.440002,14.020000,84.330002,20.510000,2.53,1.3200,58.119999,154.050003
2020-06-11,345.299988,27.000000,67.800003,12.830000,75.000000,17.840000,2.28,1.0900,52.439999,139.089996


In [15]:
simulations = 50
rng = np.random.default_rng()
budget_vec = rng.integers(low=1000, high=10000, size=simulations)
start_dt_vec = np.random.choice(data_pivot.index[:-30], size=simulations, replace=True)
col_names = ["portf_val", "cash", "init_budget"]
# df_naive = pd.DataFrame(columns = )
# df_multi = pd.DataFrame(columns = ["portf_val", "final_portf", "cash"])
naive_list = list()
multi_list = list()
multi_list2 = list()
p_list = list()
stock_price_window_list = list()
z_s = list()
z_b = list()
sell = list()
buy = list()
cash = list()

# Could parallelize this but fuck it
for i, budget in enumerate(tqdm(budget_vec)):
    try:
        stock_price_window = get_prices_window(data_clean, start_dt_vec[i])
        P_0, P_t, w_0, initial_prices, final_prices = generate_portfolios(stock_prices = stock_price_window, init_budget=budget)
        final_portfolio_value, final_portfolio, free_cash = get_naive_opt(P_0, P_t, w_0, budget, initial_prices, final_prices, print_bool=False)
        naive_list.append(pd.Series([final_portfolio_value, free_cash, budget], index = col_names))
        # final_portfolio_value, final_portfolio, free_cash = get_multi_stage_opt(P_0, P_t, w_0, budget, stock_price_window)
        # multi_list.append(pd.Series([final_portfolio_value, free_cash, budget], index = col_names))
        buy_list, sell_list, z_s_list, z_b_list, p_val_arr, cash_arr, p_arr = get_multi_stage_opt_2(P_0, P_t, w_0, budget, stock_price_window, print_bool=False, time_limit=5)
        multi_list2.append(pd.Series([p_val_arr[-1], cash_arr[-1], budget], index = col_names))
        p_list.append(p_arr)
        stock_price_window_list.append(stock_price_window)
        z_s.append(z_s_list)
        z_b.append(z_b_list)
        buy.append(buy_list)
        sell.append(sell_list)
        cash.append(cash)
        
    except Exception as e:
        print(e)
        pass
        # print(e)

df_naive = pd.concat(naive_list, axis=1).T
df_multi2 = pd.concat(multi_list2, axis=1).T

 52%|█████▏    | 26/50 [02:36<02:17,  5.73s/it]

Unable to retrieve attribute 'x'


100%|██████████| 50/50 [04:49<00:00,  5.79s/it]


In [47]:
df_naive

,portf_val,cash,init_budget
0,10462.679108,67.91002082824707,9522
1,244047.346774,68.60804696381115,6094
2,5205.799971,48.199997901916504,4780
3,5409.629978,71.68997204303741,5438
4,2180.779752,59.329758167266846,2093
5,1660.910055,30.690009117126465,1601
6,3469.203988,43.60197892189035,3132
7,1913.750016,53.7000150680542,1888
8,7811.759979,54.02995300292969,7390
9,10748.876042,59.460044145584106,9914


In [48]:
df_multi

,portf_val,cash,init_budget
0,10409.279095,67.91002082824707,9522
1,243994.126774,68.60804696381115,6094
2,5158.199971,48.199997901916504,4780
3,5354.429978,71.68997204303741,5438
4,2132.359751,59.329758167266846,2093
5,1631.160055,30.690009117126465,1601
6,3427.553988,43.60197892189035,3132
7,1862.050022,53.7000150680542,1888
8,7758.209975,54.02995300292969,7390
9,10694.556042,59.460044145584106,9914


In [49]:
df_multi2

,portf_val,cash,init_budget
0,8.009909e+06,4.898911,9522.0
1,8.843959e+05,0.181526,6094.0
2,2.720100e+04,0.820062,4780.0
3,7.350190e+03,0.985028,5438.0
4,4.068054e+05,2.195234,2093.0
5,3.443370e+03,0.569928,1601.0
6,7.373512e+03,1.511317,3132.0
7,4.259980e+03,0.289864,1888.0
8,1.235017e+04,0.281817,7390.0
9,2.373592e+04,0.438369,9914.0


In [52]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        y = df_multi.portf_val,
        x = df_multi.init_budget,
        mode = "markers",
        name = "Old Formulation",
    )
)
fig.add_trace(
    go.Scatter(
        y = df_multi2.portf_val,
        x = df_multi2.init_budget,
        mode = "markers",
        name = "New Formulation",
    )
)
fig.add_trace(
    go.Scatter(
        y = df_naive.portf_val,
        x = df_naive.init_budget,
        mode = "markers",
        name = "Naive",
    )
)

fig.update_xaxes(
    title="Starting Budget"
)

fig.update_yaxes(
    title="Multi-period Advantage",
    type="log"
)

In [58]:
pd.DataFrame(data=p_list[25], columns = stock_price_window_list[25].columns, index=stock_price_window_list[25].index)

comnam,CHEVRON CORP NEW,S L M CORP,COCA COLA EUROPEAN PARTNERS PLC,LOMAK PETROLEUM INC,DISCOVER FINANCIAL SERVICES,VANGUARD SECTOR INDEX FUNDS,DEVRY EDUCATION GROUP INC,TIDEWATER MARINE SVC INC,CANANDAIGUA WINE INC,CROWN CORK & SEAL CO INC
date,,,,,,,,,,
2021-12-08,0.0,1.0,1.0,0.0,21.0,0.0,0.0,1.0,9122.0,1.0
2021-12-09,0.0,1.0,1.0,0.0,21.0,0.0,0.0,1.0,18226.0,1.0
2021-12-10,0.0,1.0,1.0,0.0,21.0,0.0,0.0,1.0,27330.0,1.0
2021-12-13,0.0,9105.0,1.0,0.0,9125.0,0.0,0.0,1.0,18226.0,9105.0
2021-12-14,0.0,18209.0,456.0,9104.0,22.0,9104.0,0.0,9105.0,9122.0,18209.0
2021-12-15,9104.0,27313.0,9558.0,9104.0,6391.0,0.0,9104.0,18199.0,18226.0,9105.0
2021-12-16,1.0,36417.0,18662.0,18208.0,0.0,8126.0,18208.0,27303.0,27330.0,1.0
2021-12-17,9104.0,45521.0,27766.0,27312.0,0.0,8126.0,27312.0,36407.0,18226.0,1.0
2021-12-20,0.0,54625.0,36870.0,36416.0,9104.0,13687.0,36416.0,45511.0,9122.0,1.0


In [23]:
prices

comnam,GRUPO SIMEC S A B DE C V,TRAVELERS GROUP INC,MOLSON COORS BREWING CO,TRIMBLE NAVIGATION LTD,FIRST COMMONWEALTH FINANCIAL COR,DETROITBANK CORP,PUBLIC STORAGE INC,BURROUGHS CORP,ISHARES TRUST,CATERPILLAR TRACTOR INC
date,,,,,,,,,,
2020-03-11,8.6500,50.790001,-6.737501,33.230000,9.60,38.029999,220.440002,10.51,92.817333,100.690002
2020-03-12,7.1355,43.259998,40.200001,27.049999,9.21,34.950001,201.509995,8.59,88.606969,92.260002
2020-03-13,7.4500,51.040001,43.280001,28.350000,9.97,38.669998,211.339996,9.38,89.935316,99.639999
2020-03-16,7.7900,41.189999,45.245100,24.219999,8.76,31.450001,186.970001,9.50,89.127273,93.410004
2020-03-17,-6.5000,40.250000,-2.607500,22.090000,9.54,34.400002,198.360001,11.49,88.411353,100.199997
2020-03-18,7.0100,36.430000,-3.052500,21.320000,8.65,27.420000,189.960007,11.33,84.529534,100.120003
2020-03-19,-7.3800,39.639999,-3.245001,25.330000,9.07,28.950001,187.600006,11.57,83.896059,103.010002
2020-03-20,6.9394,38.060001,-4.222500,26.000000,8.79,28.930000,171.220001,10.53,84.892727,95.500000
2020-03-23,7.1600,35.389999,-7.997499,26.540001,8.20,27.340000,160.610001,10.33,86.020599,91.849998


In [19]:
df

comnam,GRUPO SIMEC S A B DE C V,TRAVELERS GROUP INC,MOLSON COORS BREWING CO,TRIMBLE NAVIGATION LTD,FIRST COMMONWEALTH FINANCIAL COR,DETROITBANK CORP,PUBLIC STORAGE INC,BURROUGHS CORP,ISHARES TRUST,CATERPILLAR TRACTOR INC
date,,,,,,,,,,
2020-03-11,3.0,0.0,1553.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0
2020-03-12,0.0,1552.0,1.0,0.0,1.0,24.0,0.0,738.0,0.0,0.0
2020-03-13,0.0,0.0,1516.0,0.0,1.0,0.0,0.0,2289.0,0.0,0.0
2020-03-16,0.0,0.0,0.0,0.0,574.0,1552.0,0.0,3840.0,0.0,0.0
2020-03-17,1552.0,0.0,1552.0,1552.0,0.0,0.0,0.0,2288.0,0.0,564.0
2020-03-18,0.0,1552.0,3104.0,3104.0,0.0,0.0,0.0,736.0,0.0,0.0
2020-03-19,1552.0,0.0,4656.0,3206.0,0.0,0.0,0.0,1.0,1000.0,0.0
2020-03-20,1552.0,0.0,6208.0,1654.0,0.0,0.0,0.0,1.0,1552.0,0.0
2020-03-23,0.0,1552.0,7760.0,102.0,0.0,1552.0,0.0,1.0,0.0,1098.0


In [22]:
portf_num = 25

fig = go.Figure()

df = pd.DataFrame(data=p_list[portf_num], columns = stock_price_window_list[portf_num].columns, index=stock_price_window_list[portf_num].index)
df_tx_b = pd.DataFrame(data=buy[portf_num], columns = stock_price_window_list[portf_num].columns, index=stock_price_window_list[portf_num].index)
df_tx_s = pd.DataFrame(data=sell[portf_num], columns = stock_price_window_list[portf_num].columns, index=stock_price_window_list[portf_num].index)
prices = stock_price_window_list[portf_num]

fig.add_trace(
    go.Scatter(
        x = df.index,
        y = df.mul(prices).sum(axis=1),
        name="Portfolio Value"
    )
)

In [26]:
fig = go.Figure()

for stock in df_tx_b.columns:
    fig.add_trace(
        go.Scatter(
            x = df_tx_b[stock].index,
            y = df_tx_b[stock].mul(prices[stock]),
            line=dict(width=1, dash='dot'),
            name=stock
        )
    )

    fig.add_trace(
        go.Scatter(
            x = df_tx_s[stock].index,
            y = -df_tx_s[stock].mul(prices[stock]),
            line=dict(width=1, dash='solid'),
            name=stock,
        )
    )

fig